# Basics of Molecular Mechanics calculations with Libra

## Table of Content <a name="TOC"></a>

1. [General setups](#setups)


2. [Setting up the interactions in a single system](#2.) 

  2.1. [Chemical system & force field creation](#2.1.)
  
  2.2. [Atomistic Hamiltonian setup](#2.2.)
  
     * 2.2.1. [Simple example](#2.2.1.)
     
     * 2.2.2. [Change the functional and recompute energy](#2.2.2.)
    

3. [Computing energies of multiple systems](#3.)   


4. [Mixing different force fields](#4.) 



### A. Learning objectives

- to conduct classical molecular mechanics calculations with Libra (energies and forces)
- to learn about the hierarchies, inheritances, and relationships of the Hamiltonian classes
- to show the statistics of interactions in a system with a given force field
- to  combine two force fields

### B. Use cases

- molecular mechanics calculations (energy & forces)
- changing the force fields
- mixing force fields
- classical molecular dynamics with force fields


### C. Functions

- `libra_py`  
  - `nve_md`
    - [`nve_md_init`](#nve_md_init-1)
    - [`nve_md_step`](#nve_md_step-1)
          

### D. Classes and class members

- `liblibra::libchemobjects`
  - `libchemsys`
    - `System`
      - [`init_atom_velocities`](#init_atom_velocities-1)        
      - [`set_atomic_q`](#set_atomic_q-1)
      
- `liblibra::libdyn`
  - `libelectronic`
    - `Electronic`
      - [`Electronic`](#Electronic-1)
    
  - `libnuclear`
    - `Nuclear`
      - [`q`](#q-1)
      - [`Nuclear`](#Nuclear-1)          

- `liblibra::libhamiltonian`
  - `libhamiltonian_atomistic`
    - `libhamiltonian_mm`
      - `Hamiltonian_MM`
        - [`Hamiltonian_MM`](#Hamiltonian_MM-1)
      - `listHamiltonian_MM`
        - [`active_interactions`](#active_interactions-1)
        - [`interactions`](#interactions-1)
        - [`is_active`](#is_active-1)
        - [`is_new_interaction`](#is_new_interaction-1)
        - [`listHamiltonian_MM`](#listHamiltonian_MM-1)      
    - `Hamiltonian_Atomistic`
      - [`compute`](#compute-1)
      - [`Hamiltonian_Atomistic`](#Hamiltonian_Atomistic-1)
      - [`set_Hamiltonian_type`](#set_Hamiltonian_type-1)
      - [`set_interactions_for_atoms`](#set_interactions_for_atoms-1)
      - [`set_system`](#set_system-1)
      - [`show_interactions`](#show_interactions-1)
      - [`show_interactions_statistics`](#show_interactions_statistics-1)
  - `libhamiltonian_generic`
    - `Hamiltonian`    
      - [`compute`](#compute-1)
      - [`H`](#H-1)
      - [`dHdq`](#dHdq-1)      
      

## 1. General setups
<a name="setups"></a>[Back to TOC](#TOC)

Let's import all the needed libraries, including the `py3Dmol` library for visualizing the structures we generate.

In [1]:
import math
import liblibra_core
from liblibra_core import *
from libra_py import units
from libra_py import LoadPT, LoadMolecule
from libra_py import LoadUFF, LoadGAFF, LoadTRIPOS, LoadMMFF94
from libra_py import nve_md

import os, sys
#stdout = sys.stdout

import py3Dmol
import matplotlib.pyplot as plt
#from matplotlib.pyplot import figure
%matplotlib inline

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::v

## 2. Setting up the interactions in a single system
<a name="2."></a>[Back to TOC](#TOC)

By now, we should already know:

- how to create a chemical system
- how to create a force field object

So, now we want to combine the two components to define the interactions in a molecular system and compute its energy and atomic forces.

### 2.1. Chemical system & force field creation
<a name="2.1."></a>[Back to TOC](#TOC)

In [2]:
# Create a Universe and populate it
U = Universe()
verbose = 0
LoadPT.Load_PT(U, "elements.dat", verbose)


#======= System ==============
syst = System()

# Choose any of the systems you want to try
#Load_Molecule(U, syst, os.getcwd()+"/Clusters/2benz.ent", "pdb")
LoadMolecule.Load_Molecule(U, syst, os.getcwd()+"/Molecules/test1a.pdb", "pdb_1")
syst.determine_functional_groups(0)  # 

# Printing info about the system
syst.show_fragments()
syst.show_molecules()
print("Number of atoms in the system = ", syst.Number_of_atoms)


#======= Parameters ==============
# Create force field objects
uff = ForceField({"bond_functional":"Harmonic",
                  "angle_functional":"Harmonic",
                  "vdw_functional":"LJ12_6"})

# Load parameters
LoadUFF.Load_UFF(uff)

# Set up functional forms
#uff.set_functionals({"bond":"Harmonic","angle":"Harmonic","vdw":"LJ12_6"})


24
Number of atoms in the system =  11


### 2.2. Atomistic Hamiltonian setup
<a name="2.2."></a>[Back to TOC](#TOC)

#### 2.2.1. Simple example
<a name="2.2.1."></a>[Back to TOC](#TOC)

Now we can focus on the new stuff - let's create an object of the `Hamiltonian_Atomistic`. Its constructor takes the number of electronic (1) and nuclear (`3*syst.Number_of_atoms` ) DOFs as the input. In this case, we are interested in an adiabatic dynamics, so there is only  a single electronic DOF involved.

<a name="ham_dia-1"></a><a name="ham_adi-1"></a><a name="d1ham_dia-1"></a><a name="d1ham_adi-1"></a>
This operation simply allocates memory for various internal variables inherided from the parent `Hamiltonian` class, such as:

- `ham_dia` to store diabatic electronic Hamiltonian for N states
- `ham_adi` to store adiabatic electronic Hamiltonian for N states
- `d1ham_dia` and `d1ham_adi` to store the derivatives of these Hamiltonians w.r.t. nuclear DOFs

<a name="Hamiltonian_Atomistic-1"></a>

In [3]:
ham = Hamiltonian_Atomistic(1, 3*syst.Number_of_atoms)

We also set up the Hamiltonian to be of the MM type. In principle, one could use the `Hamiltonian_Atomistic` object  to define a quantum Hamiltonian.
<a name="set_Hamiltonian_type-1"></a>

In [4]:
# Choose the Hamiltonian to be of MM type
ham.set_Hamiltonian_type("MM")

<a name="listHamiltonian_MM-1"></a>
This operation allocates memory to hold the instances of the `listHamiltonian_MM` class, which is just a vector of the `Hamiltonian_MM` class objects.

<a name="Hamiltonian_MM-1"></a>
The list `Hamiltonian_MM` class represents a collection of classical interactions in a molecular (or solid-state) system. This is essentially a classical-mechanical Hamiltonian. Although it is called a listHamiltonian - this is only for the uniformity with QM Hamiltonian and also for better flexibility - this way, the actual Hamiltonian can be a multi-resolution Hamiltonian, so each component can be tackeld by different MM level of theory.

<a name="interactions-1"></a>
The key element hidden in the `listHamiltonian_MM` class is the list of interaction objects. 

    vector<Hamiltonian_MM> interactions;  
    
Each interaction object describes an interaction of a certain type and for a given set of atoms. For instance, it could be an interaction of the bonded type between just two atoms or it could be an Ewald summation for all atoms in the system. Each interaction object comes with its own set of parameters, which are set up according to the force fields used. Thus, one can mix and match multiple force fields and customize how to treat each set of atoms.


<a name="active_interactions-1"></a>
One can disable some interactions (so no calculations is conducted for them). The status of the interactions is stored in the `active_interactions` list which contains the indices of the active interactions. 

    vector<int>     active_interactions;  

<a name="is_new_interaction-1"></a>
When a new interaction is added to the `listHamiltonian_MM` object, it is checked that this interaction is unique and it hadn't been included in the object before. This is done with the `is_new_interaction` function.

    int is_new_interaction(Hamiltonian_MM&);

<a name="is_active-1"></a>
One can also use the pointers to the `Atom` objects to check whether the interaction between given atoms is active. This is done with the `is_active` function:

    bool is_active(Atom&,Atom&);
    bool is_active(Atom&,Atom&,Atom&);
    bool is_active(Atom&,Atom&,Atom&,Atom&);


Just to recap, here is the structure of inheritance and relationships:

![](Hamiltonians.png)

We can then partition the atoms in the system to several lists and define interactions within each subset of atoms according to the desired force field. For instance, for  a nanocar moving on a metal surface one could define the nanocar itself to  be described by the UFF force field, while the metal could be handled by other potentials such as MEAM (not implemented here). Finally, one would also need to define the interactions of the atoms belonging to different sub-systems according to yet another force field model.

In this example, we will select all atoms as the interaction region to keep it simple. Note that the range of the indies corresponds to the Atomic IDs (starting from 1), not  indices. 

In [5]:
# Define regions for different levels of theory
atlst1 = list( range(1,syst.Number_of_atoms+1) )

<a name="set_interactions_for_atoms-1"></a>
The interactions are defined using the `set_interactions_for_atoms` function. The function uses information about connectivities in the given system and the types of interactions and their parameters as defined by the force field object. In the result, the interactions are stored within the `ham` object.

<a name="show_interactions_statistics-1"></a>
With the help of  `show_interactions_statistics`  function  we can learn how many and what kind of interactionas are defined for our system:

In [6]:
# Create interactions between atoms from atom list 1
verb = 0
assign_rings = 0

# Set atomistically-resolved (as opposed to coarse-grained resolutions) interactions
# among all atoms in group atlst1 with all atoms in the same list
ham.set_interactions_for_atoms(syst, atlst1, atlst1, uff, verb, assign_rings)    # The simplest sintax

# Show setup interactions
ham.show_interactions_statistics()


<a name="set_system-1"></a>
Finally, we associate the Hamiltonian object with the system object using the `set_system` function. This makes all the Hamiltonian to know about all atomic displacements.

<a name="compute-1"></a>
We are now ready to compute system's energy by calling the `compute` function. The energy is stored in the system's Hamiltonian matrix of the dimensions N x N, where N is the number of electronic states. 

<a name="H-1"></a>
To access the energy of i-th state, we can simply call the `H` function with the parameters `(i, i)`

<a name="dHdq-1"></a>
This calculation also populates the gradients of the energy with respect to all nuclear DOFs. The computed properties are accessed via the `dHdq` function with the parameters `(i,i, idof)`, where `idof` is the index of the nuclear degree of freedom of interest.  Here, they are organized as follows: $x_1, y_1, z_1, x_2, y_2, z_2, ... x_i, y_i, z_i$


In [7]:
ham.set_system(syst)
ham.compute()
print( "Energy = ", ham.H(0,0), " a.u.")
print( "Gradient 1 = ", ham.dHdq(0,0,0), " a.u.")
print( "Gradient 3 = ", ham.dHdq(0,0,3), " a.u.")


Energy =  (0.007140292248010003+0j)  a.u.
Gradient 1 =  (-0.012518643094809252+0j)  a.u.
Gradient 3 =  (-0.0072832662728009225+0j)  a.u.


#### 2.2.2. Change the functional and recompute energy
<a name="2.2.2."></a>[Back to TOC](#TOC)

Here, let's change the functional - remove the bonded interactions and add dihedrals - and recompute the energy. We can see that the remaining terms contribute very little to the current energy value. The main contribution was due to bond stretching.

**IMPORTANT:** one has to use new System, Hamiltonian, and ForceField objects

In [8]:
# System
syst1 = System()
LoadMolecule.Load_Molecule(U, syst1, os.getcwd()+"/Molecules/test1a.pdb", "pdb_1")
syst1.determine_functional_groups(0)  # 

# force field
uff1 = ForceField({ "dihedral_functional":"General0", "vdw_functional":"LJ", 
                    "angle_functional":"Fourier", "R_vdw_on":0.0, "R_vdw_off":2.0 })
LoadUFF.Load_UFF(uff1)

# New Hamiltonian
ham1 = Hamiltonian_Atomistic(1, 3*syst.Number_of_atoms)
ham1.set_Hamiltonian_type("MM")
ham1.set_interactions_for_atoms(syst1, atlst1, atlst1, uff1, verb, assign_rings)    # The simplest sintax
ham1.show_interactions_statistics()

# Compute energy
ham1.set_system(syst1)
ham1.compute()
print( "Energy = ", ham1.H(0,0), " a.u.")
print( "Gradient 1 = ", ham1.dHdq(0,0,0), " a.u.")
print( "Gradient 3 = ", ham1.dHdq(0,0,3), " a.u.")

24
Energy =  (2.749343352645219e-07+0j)  a.u.
Gradient 1 =  (-1.914604313984184e-05+0j)  a.u.
Gradient 3 =  (1.7723407068891845e-05+0j)  a.u.


## 3. Computing energies of multiple systems
<a name="3."></a>[Back to TOC](#TOC)

Now let's apply the above recipe (in a more condensed form) to several molecules.

In [9]:
# Create Universe and populate it
U = Universe()
verbose = 0
LoadPT.Load_PT(U, "elements.dat", verbose)

# Create force field
uff = ForceField({"bond_functional":"Harmonic","angle_functional":"Fourier",
                   "vdw_functional":"LJ","dihedral_functional":"General0",
                    "R_vdw_on":6.0,"R_vdw_off":7.0}
                )
LoadUFF.Load_UFF(uff)
verb = 0
assign_rings = 1


#======= System ==============
for i in range(1,13):
#for i in [1]:
    print( "=================== System ",i,"=======================")

    syst = System()
    LoadMolecule.Load_Molecule(U, syst, os.getcwd()+"/Molecules/test"+str(i)+"a.pdb", "pdb_1")
    syst.determine_functional_groups(1)  # 
    syst.show_atoms()
    syst.show_fragments()
    syst.show_molecules()

    syst.print_xyz("molecule"+str(i)+".xyz",1)

    print( "Number of atoms in the system = ", syst.Number_of_atoms)
    atlst1 = list(range(1,syst.Number_of_atoms+1))


    # Creating Hamiltonian
    ham = Hamiltonian_Atomistic(1, 3*syst.Number_of_atoms)
    ham.set_Hamiltonian_type("MM")
    ham.set_interactions_for_atoms(syst, atlst1, atlst1, uff, verb, assign_rings)  
    ham.show_interactions_statistics()


    ham.set_system(syst)
    ham.compute()
    print( "Energy = ", ham.H(0,0), " a.u.")
    print( "Gradient 1 = ", ham.dHdq(0,0,0), " a.u.")
    print( "Gradient 3 = ", ham.dHdq(0,0,3), " a.u.")




=================== System  1 =======================
24
Number of atoms in the system =  11
Energy =  (0.007140324859244774+0j)  a.u.
Gradient 1 =  (-0.012520301102323814+0j)  a.u.
Gradient 3 =  (-0.007282875709903818+0j)  a.u.
=================== System  2 =======================
13
Number of atoms in the system =  6
Energy =  (0.0075670334465485975+0j)  a.u.
Gradient 1 =  (0.04140552810123973+0j)  a.u.
Gradient 3 =  (-0.004668040251792126+0j)  a.u.
=================== System  3 =======================
35
Number of atoms in the system =  17
Energy =  (0.033609462499328675+0j)  a.u.
Gradient 1 =  (0.027346563854722887+0j)  a.u.
Gradient 3 =  (-0.02647619264176351+0j)  a.u.
=================== System  4 =======================
15
Number of atoms in the system =  7
Energy =  (0.01078361381883055+0j)  a.u.
Gradient 1 =  (-0.021782762154656025+0j)  a.u.
Gradient 3 =  (-0.00721016192172529+0j)  a.u.
=================== System  5 =======================
23
Number of atoms in the system =  1

## 4. Mixing different force fields
<a name="4."></a>[Back to TOC](#TOC)

Here is one way to set up a force field: create and empty object and then use the `set_functional` function:
<a name="show_interactions-1"></a>

In [10]:
# Create Universe and populate it
U = Universe(); LoadPT.Load_PT(U, "elements.dat")

# Create force field with only bond interactions
uff = ForceField({"bond_functional":"Harmonic"}) #, "mb_functional":"LJ_Coulomb","R_vdw_on":10.0,"R_vdw_off":15.0 })
LoadUFF.Load_UFF(uff,"uff_new.dat")


Xff = ForceField({"angle_functional":"Fourier"})  # no FF name, so we'll rely on the manually-defined atom type    

class atomrecord:
    pass

ff = atomrecord()
ff.Atom_ff_int_type = 1
ff.Atom_ff_type = "H_"
atom_record = Atom_Record()
atom_record.set(ff)
Xff.Add_Atom_Record(atom_record)

ff = atomrecord()
ff.Atom_ff_int_type = 2
ff.Atom_ff_type = "H_b"
atom_record = Atom_Record()
atom_record.set(ff)
Xff.Add_Atom_Record(atom_record)

class anglerecord:
    pass

ff = anglerecord()
ff.Atom1_ff_type    = "H_"
ff.Atom2_ff_type    = "H_b"
ff.Atom3_ff_type    = "H_b"
ff.Angle_k_angle    = 100.0
ff.Angle_theta_eq   = 120.0
angle_record = Angle_Record()
angle_record.set(ff)
angle_record.show_info()
Xff.Add_Angle_Record(angle_record)

ff = anglerecord()
ff.Atom1_ff_type    = "H_b"
ff.Atom2_ff_type    = "H_b"
ff.Atom3_ff_type    = "H_b"
ff.Angle_k_angle    = 100.0
ff.Angle_theta_eq   = 120.0
angle_record = Angle_Record()
angle_record.set(ff)
angle_record.show_info()
Xff.Add_Angle_Record(angle_record)

Xff.show_atom_records()
Xff.show_angle_records()

# Create molecular system and initialize the properties
syst = System()
LoadMolecule.Load_Molecule(U, syst, "Molecules/Hn.ent", "pdb")
syst.determine_functional_groups(0)  # do not assign rings
syst.init_fragments()
print("Number of atoms in the system = ", syst.Number_of_atoms)
atlst1 = list(range(1,syst.Number_of_atoms+1))


# Creating Hamiltonian and initialize it
ham = Hamiltonian_Atomistic(1, 3*syst.Number_of_atoms)
ham.set_Hamiltonian_type("MM")
ham.set_interactions_for_atoms(syst, atlst1, atlst1, uff, 1, 0)  # 0 - verb, 0 - assign_rings
ham.show_interactions_statistics()
ham.show_interactions()


#======= Now here, we'll add extra interactions due another FF ============
# keep in mind that the FF atom types in syst are already set to those of UFF
# so, we don't need to define extra types
#
# Also, we need to modify the coordination of all atoms

for i in range(syst.Number_of_atoms):
    syst.Atoms[i].Atom_coordination = 1
    syst.Atoms[i].is_Atom_coordination = 1

ham.set_interactions_for_atoms(syst, atlst1, atlst1, Xff, 1, 0)  # 0 - verb, 0 - assign_rings    
ham.show_interactions_statistics() 
ham.show_interactions()

# Bind Hamiltonian and the system   
ham.set_system(syst);   

# Compute and get energy
ham.compute();   
print("Energy = ", ham.H(0,0), " a.u.")



22
Number of atoms in the system =  10
Energy =  (0.1202969943081751+0j)  a.u.


Although it is a bit early to do the MD, the FF mixing example is best demonstrated with some MD: run the code below with and without the Xff added, visualize the MD trajectories produced in both cases. Can you see the difference?

To turn off the Xff contribution, just comment the following line in the above cell:

    ham.set_interactions_for_atoms(syst, atlst1, atlst1, Xff, 1, 0)  # 0 - verb, 0 - assign_rings    

<a name="nve_md_init-1"></a><a name="Electronic-1"></a><a name="Nuclear-1"></a><a name="init_atom_velocities-1"></a><a name="set_atomic_q-1"></a><a name="q-1"></a><a name="nve_md_step-1"></a>

In [11]:
do_md = 1
if do_md:
    
    rnd = Random()
    
    # Electronic DOFs
    el = Electronic(1,0)

    # Nuclear DOFs
    mol = Nuclear(3*syst.Number_of_atoms)

    # Initialize MD variables
    nve_md.nve_md_init(syst, mol, el, ham)
    
    ########################## Production MD #################################

    syst.init_atom_velocities(300.0, rnd)

    f = open("_en_md.txt","w"); f.close()
        
    params = {"dt":20.0, "integrator":"DLML" }

    for i in range(100):
        syst.set_atomic_q(mol.q)
        syst.print_xyz("_mol_md.xyz",i)

        for j in range(250):
            ekin, epot, etot = nve_md.nve_md_step(syst, mol, el, ham, params )

        syst.cool()

        f = open("_en_md.txt","a")
        f.write("i= %3i ekin= %8.5f  epot= %8.5f  etot= %8.5f\n" % (i, ekin, epot, etot))
        f.close()


For the impatient, the results shall look like:

|  with xff |   without xff |
| ----- | ----- |
| ![](with_xff.png)  | ![](without_xff.png)  |